In [1]:
import sqlite3
import json
db_file = "./gptitor.db"

In [50]:
user_id=1

In [124]:
select_query = '''
    SELECT round(AVG(json_extract(f.metrics, '$.criterion_1')), 1) AS avg_criterion_1, 
        round(AVG(json_extract(f.metrics, '$.criterion_2')), 1) AS avg_criterion_2,
        round(AVG(json_extract(f.metrics, '$.criterion_3')), 1) AS avg_criterion_3,
        round(AVG(json_extract(f.metrics, '$.criterion_4')), 1) AS avg_criterion_4
    FROM 
        messages m
    JOIN 
        conversations c ON m.conversation_id = c.conversation_id
    JOIN 
        feedback f ON m.feedback_id = f.feedback_id
    WHERE 
        c.user_id = ?
        AND m.message_class = 'Request';
    '''

In [125]:
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute(select_query, (user_id,))
data = cursor.fetchone()

conn.close()

In [126]:
data

(None, None, None, None)

In [104]:
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any, List
from datetime import datetime

class Metrics(BaseModel):
    metrics: Dict[str, Any] = Field(
        default={
            "criterion_1": 0.0,
            "criterion_2": 0.0,
            "criterion_3": 0.0,
            "criterion_4": 0.0
        },
        examples=[{
            "criterion_1": 5,
            "criterion_2": 4.5,
            "criterion_3": 3.2,
            "criterion_4": 0.0
        }]
    )

In [105]:
json_data = {
    "criterion_1": data[0],
    "criterion_2": data[1],
    "criterion_3": data[2],
    "criterion_4": data[3]
}
json_data

{'criterion_1': 4.2,
 'criterion_2': 3.0,
 'criterion_3': 3.1,
 'criterion_4': 3.5}

In [106]:
Metrics(metrics=json_data)

Metrics(metrics={'criterion_1': 4.2, 'criterion_2': 3.0, 'criterion_3': 3.1, 'criterion_4': 3.5})

In [108]:
select_query = '''
SELECT
    COUNT(DISTINCT c.conversation_id) AS total_conversations,
    COUNT(m.message_id) AS total_request_messages
FROM
    conversations c
LEFT JOIN
    messages m ON c.conversation_id = m.conversation_id
WHERE
    c.user_id = ?
    AND m.message_class = 'Request';
'''

In [118]:
metrics = Metrics()
metrics.metrics

{'criterion_1': 0.0,
 'criterion_2': 0.0,
 'criterion_3': 0.0,
 'criterion_4': 0.0}

In [121]:
select_query = ''' SELECT
    COUNT(DISTINCT c.conversation_id) AS total_conversations,
    COUNT(m.message_id) AS total_messages,
    COUNT(DISTINCT m.task_id) AS task_ids
FROM
    conversations c
JOIN
    messages m ON c.conversation_id = m.conversation_id
WHERE
    c.user_id = ?
    AND m.message_class = 'Request';
'''

In [129]:
class PersonalStatistics(BaseModel):
    metrics: Dict[str, Any] = Field(
        default={
            "criterion_1": 0.0,
            "criterion_2": 0.0,
            "criterion_3": 0.0,
            "criterion_4": 0.0
        },
        examples=[{
            "criterion_1": 5,
            "criterion_2": 4.5,
            "criterion_3": 3.2,
            "criterion_4": 0.0
        }]
    )
    activity: Dict[str, Any] = Field(
        default={
            "total_queries": 0,
            "total_conversations": 0,
            "tasks_solved": 0
        },
        examples=[{
            "total_queries": 10,
            "total_conversations": 15,
            "tasks_solved": 3
        }]
    )

In [139]:
select_query = """SELECT feedback.feedback_id, feedback.comment, feedback.metrics
                        FROM feedback
                        JOIN messages ON feedback.feedback_id = messages.feedback_id
                        WHERE messages.message_id = ?"""

In [140]:
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute(select_query, (1,))
data = cursor.fetchone()

conn.close()

In [137]:
class Feedback(BaseModel):
    feedback_id: int = Field(default=0, 
                             examples=[0])
    comment: str = Field(default="", 
                         examples=["Example comment"], 
                         max_length=2048)
    metrics: Metrics = Field(default={"criterion_1": 0.0,
                                      "criterion_2": 0.0,
                                      "criterion_3": 0.0,
                                      "criterion_4": 0.0})  

In [141]:
metrics_feedback = Metrics(metrics=json.loads(data[2]))
metrics_feedback

Metrics(metrics={'criterion_1': 4.7, 'criterion_2': 1.5, 'criterion_3': 4.0, 'criterion_4': 0.5})

In [142]:
Feedback(metrics=metrics_feedback)

Feedback(feedback_id=0, comment='', metrics=Metrics(metrics={'criterion_1': 4.7, 'criterion_2': 1.5, 'criterion_3': 4.0, 'criterion_4': 0.5}))

In [127]:
activity = {
            "total_queries": data[0],
            "total_conversations": data[1],
            "tasks_solved": data[2]
        }

In [128]:
activity

{'total_queries': None, 'total_conversations': None, 'tasks_solved': None}

In [149]:
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
sql_select = """SELECT feedback.metrics
                        FROM feedback
                        JOIN messages ON feedback.feedback_id = messages.feedback_id
                        WHERE messages.message_id = ?"""
        
cursor.execute(sql_select, (1,))
metrics = cursor.fetchone()
json.loads(metrics[0])

{'criterion_1': 4.7,
 'criterion_2': 1.5,
 'criterion_3': 4.0,
 'criterion_4': 0.5}

In [146]:
feedback

Feedback(feedback_id=2, comment=' "Calculate the distance traveled in kilometers and the total number of days for a road trip of 2300 kilometers at an average speed of 80 kilometers per hour, with daytime stops of 4 hours each day. Also, calculate the probability that the total number of days for the trip is correct."\n    Answer: This query is clear and specific, providing all necessary information to calculate the distance traveled, the total number of days for the road trip, and the probability of the answer being correct. It is well-structured and easy to understand.\n\n    Now, please formulate the query to the language model to solve this problem.\n    Answer: "Calculate the distance traveled in kilometers and the total number of days for a road trip of 2300 kilometers at an average speed of 80 kilometers per hour, considering daytime stops of 4 hours each day. Additionally, determine the probability that the calculated total number of days for the trip is accurate."', metrics=Me